In this notebook we try to explore the frequency of the articles published within the time span of 2019/03-2024/03 using Apple.

First, we look at the daily news article counts, and the monthly article counts. Then we explore some days where the article count was very low or very high. In the instance with the highest article count, we take a breif look at the movement of the stock price around that date.

Key observations: 
1. While there seem to be at least one article reporting on stock movements each day, these articles seem to vary in tone depending on other market information around the day.
2. Perhaps there are instances where the combined sentiment score for the day is hiding a lot of important details.  For example, some days could have a majority of neutral or positive toned articles with a one or two significantly negative toned articles. In such an example the combined score may seem relatively neutral, but the fact that there are negatively toned articles (whereas perhaps most days this ticker may only get neutral or postive news), may still skew readers' perception in a negative direction.

SECTION 1: APPLE

In [1]:
import pandas as pd
import numpy as np
import datetime
from seaborn import set_style
import plotly as py

set_style("whitegrid")

In [2]:
news = pd.read_csv('../data/news_data_2019_2024_complete.csv')

In [3]:
#restricting to specific tickers
Apl = news.loc[news["Ticker"] == 'AAPL'].reset_index()
Msft = news.loc[news["Ticker"] == 'MSFT'].reset_index()
Amz = news.loc[news["Ticker"] == 'AMZN'].reset_index()
Ggl = news.loc[news["Ticker"] == 'GOOGL'].reset_index()
Nvd = news.loc[news["Ticker"] == 'NVDA'].reset_index()

In [4]:
Apl.head()

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
0,1,1,2019-03-15 10:47:26+00:00,2019-03-15,10:47:26,AAPL,Technology,The Motley Fool,0.0,0.748,0.252,0.5248,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,https://www.fool.com/investing/2019/03/14/dont...
1,11,11,2019-03-15 19:16:44+00:00,2019-03-15,19:16:44,AAPL,Technology,CNBC,0.0,0.651,0.349,0.9493,Apple's new streaming service seeks Oscar glory,Apple is reportedly hiring strategists to help...,https://www.cnbc.com/2019/03/15/apples-new-str...
2,15,15,2019-03-15 19:40:00+00:00,2019-03-15,19:40:00,AAPL,Technology,The Motley Fool,0.0,1.000,0.000,0.0000,Apple Just Scored the $1 Billion It Was Asking...,But it won't be able to collect quite yet.,https://www.fool.com/investing/2019/03/15/appl...
3,22,22,2019-03-18 14:54:26+00:00,2019-03-18,14:54:26,AAPL,Technology,CNBC,0.0,0.912,0.088,0.4019,Apple unveils new iPad Air and iPad mini ahead...,The upgraded models offer keyboard support for...,https://www.cnbc.com/2019/03/18/apple-unveils-...
4,23,23,2019-03-18 15:29:38+00:00,2019-03-18,15:29:38,AAPL,Technology,Seeking Alpha,0.0,0.812,0.188,0.7717,Buy Apple For Great Total Return And Future 10...,This article is about Apple and why it's a buy...,https://seekingalpha.com/article/4249322-buy-a...


In [5]:
Apl_c = Apl.groupby('Date')['Ticker'].count().reset_index()

In [6]:
Apl_counts = Apl_c.rename(columns={"Ticker": "Article_Count" })


In [7]:
Apl_counts.head()

,Date,Article_Count
0,2019-03-15,3
1,2019-03-18,8
2,2019-03-19,10
3,2019-03-20,8
4,2019-03-21,12


Let's look at a histogram of "article count per day"

In [8]:
import plotly.express as px
#df = px.data.tips()
fig = px.histogram(Apl_counts, x="Article_Count",  labels={"Article_Count": "Article Count Per Day"})
fig.update_layout(yaxis_title="Days")
fig.show()

Let's look at Article Counts per Month

In [9]:
#Create a column for month
Apl_counts = Apl_counts.assign(month = Apl_counts.Date.str[0:7])

In [10]:
Apl_counts.tail()

,Date,Article_Count,month
1661,2024-03-11,7,2024-03
1662,2024-03-12,10,2024-03
1663,2024-03-13,13,2024-03
1664,2024-03-14,7,2024-03
1665,2024-03-15,12,2024-03


In [51]:
#create another dataframe containing article count by month
Apl_counts_month = Apl_counts.groupby('month')['Article_Count'].sum().reset_index()

In [52]:
Apl_counts_month.head()

,month,Article_Count
0,2019-03,115
1,2019-04,181
2,2019-05,192
3,2019-06,264
4,2019-07,247


In [53]:
fig = px.histogram(Apl_counts_month, x="Article_Count", labels={"Article_Count" : "Article Count per Month"})
fig.update_layout(yaxis_title="Months")
fig.show()

Let's look at this from a different angle. How about a simple graph that plots article counts against date.

In [14]:
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=Apl_counts['Date'], y=Apl_counts['Article_Count'])])
fig.show()


It seems there are lots of days where the article count is really low. Let's see how many days there are with 0 articles about Apple.

In [15]:
#number of days with 0 articles
Apl_counts.loc[Apl_counts["Article_Count"] == 0].count()

Date             0
Article_Count    0
month            0
dtype: int64

Apple really generates regular buzz, huh? Either that or, maybe there are just obligatory daily stock movement reports that we should perhaps(?) filter out. (I suppose we'll get more context when we explore other tickers)

Let's see what the minimum article count for a day is.

In [16]:
Apl_counts["Article_Count"].min()

1

Alright, then let's see how many days we have only one article.

In [17]:
Apl_counts.loc[Apl_counts["Article_Count"] == 1].count()

Date             206
Article_Count    206
month            206
dtype: int64

In [18]:
#double checking the total article count
#Apl_counts_month["Article_Count"].sum()

Let's look at a date with just one article

In [19]:
Apl.loc[Apl["Date"] == "2020-08-29"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
4363,17593,17593,2020-08-29 15:31:54+00:00,2020-08-29,15:31:54,AAPL,Technology,Zacks Investment Research,0.0,1.0,0.0,0.0,Why Is Apple (AAPL) Up 29.8% Since Last Earnin...,Apple (AAPL) reported earnings 30 days ago. Wh...,https://www.zacks.com/stock/news/1049587/why-i...


As presumed, it is a report on stock movement (and interestingly it has a neutral sentiment score). Let's look at another one.

In [20]:
Apl.loc[Apl["Date"] == "2020-10-21"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
4794,19692,19692,2020-10-21 17:21:25+00:00,2020-10-21,17:21:25,AAPL,Technology,InvestorPlace,0.057,0.775,0.168,0.7845,Greater Than Expected iPhone 12 Demand Makes A...,"AAPL stock could take off again, if preliminar...",https://investorplace.com/2020/10/greater-than...


In [21]:
#not sure how to print the entire headline
df = Apl.loc[Apl["Date"] == "2020-10-21"]
print(df.get("Headline"))

4794    Greater Than Expected iPhone 12 Demand Makes A...
Name: Headline, dtype: object


Huh, that one is reporting on stock movement. But it contains additional commentary on the products as well. So if we were planning on throwing out articles reporting simply on stock movement, this situation is quite tangled. This is very much something we should keep. Interestingly enough, unlike the previous article, this one does not display neutral sentiment.

Maybe we should look at a few others.

In [22]:
Apl.loc[Apl["Date"] == "2023-10-28"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
11966,58033,58033,2023-10-28 12:38:00+00:00,2023-10-28,12:38:00,AAPL,Technology,The Motley Fool,0.043,0.834,0.123,0.4497,Can Apple Stock Double in 5 Years? Here's What...,Apple shares have tripled in the past five yea...,https://www.fool.com/investing/2023/10/28/can-...


This one seems "mostly" neutral

In [23]:
Apl.loc[Apl["Date"] == "2022-07-03"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
8828,40704,40704,2022-07-03 11:30:04+00:00,2022-07-03,11:30:04,AAPL,Technology,InvestorPlace,0.0,0.77,0.23,0.861,The Next Apple Product Launch Paves the Way fo...,The best way to play a new Apple product launc...,https://investorplace.com/hypergrowthinvesting...


High neutral score. But no negativity at all, pushing the compound score to be high.

It seems that these "one per day" obligatory stock movement reports can't be thrown out altogether. The way these are reported, still can impact the next day's price movements.

There aren't matter of fact reports. The writer's optimism or pessimism impacted by other market factors may influence the writing and hence the reader's attitude.

Before we move onto other tickers, let's look at the date with the highest article count as well.

In [24]:
Apl_counts["Article_Count"].max()

63

In [25]:
Apl_counts.loc[Apl_counts["Article_Count"] == 63]

,Date,Article_Count,month
161,2019-09-10,63,2019-09


In [26]:
Apl.loc[Apl["Date"] == "2019-09-10"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
1308,4917,4917,2019-09-10 00:27:00+00:00,2019-09-10,00:27:00,AAPL,Technology,CNBC,0.000,1.000,0.000,0.0000,Apple's new iPhones might not be able to charg...,The revised prediction indicates that Apple mi...,https://www.cnbc.com/2019/09/09/apple-2019-iph...
1309,4918,4918,2019-09-10 00:45:00+00:00,2019-09-10,00:45:00,AAPL,Technology,The Motley Fool,0.000,1.000,0.000,0.0000,Apple's Big Event: What to Watch,Here's what investors will be watching during ...,https://www.fool.com/investing/2019/09/09/appl...
1310,4922,4922,2019-09-10 12:00:00+00:00,2019-09-10,12:00:00,AAPL,Technology,The Motley Fool,0.112,0.736,0.152,0.0953,Why Apple's In-Screen Fingerprint Reader Could...,Tech companies are collecting more information...,https://www.fool.com/investing/2019/09/10/why-...
1311,4923,4923,2019-09-10 12:51:04+00:00,2019-09-10,12:51:04,AAPL,Technology,Reuters,0.000,0.942,0.058,0.3182,Apple to reveal streaming service prices while...,Apple Inc is set on Tuesday to announce prici...,https://www.reuters.com/article/us-apple-iphon...
1312,4928,4928,2019-09-10 13:40:00+00:00,2019-09-10,13:40:00,AAPL,Technology,Zacks Investment Research,0.000,0.837,0.163,0.6808,Can Smartphone Market Rebound in 2020? AAPL & ...,"Per IDC, the global smartphone market witnessi...",https://www.zacks.com/stock/news/511071/can-sm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,5009,5009,2019-09-10 21:25:00+00:00,2019-09-10,21:25:00,AAPL,Technology,The Motley Fool,0.087,0.913,0.000,-0.2960,Apple Bolsters Its Fast-Growing Wearables Busi...,The tech giant has no intention of slowing dow...,https://www.fool.com/investing/2019/09/10/appl...
1367,5010,5010,2019-09-10 21:30:43+00:00,2019-09-10,21:30:43,AAPL,Technology,The Guardian,0.033,0.808,0.158,0.6690,iPhone 11: Apple's most ambitious bid yet to c...,iPhone 11 Pro is being touted as ‘the most adv...,https://www.theguardian.com/technology/2019/se...
1368,5011,5011,2019-09-10 21:40:10+00:00,2019-09-10,21:40:10,AAPL,Technology,Market Watch,0.000,0.945,0.055,0.3384,MarketWatch First Take: Apple iPhone event rev...,The most surprising new feature Apple Inc. deb...,https://www.marketwatch.com/story/apple-iphone...
1369,5012,5012,2019-09-10 21:59:00+00:00,2019-09-10,21:59:00,AAPL,Technology,CNBC,0.000,1.000,0.000,0.0000,Apple unveils the iPhone 11 – three experts on...,Apple unveiled its latest smartphone and a ran...,https://www.cnbc.com/2019/09/10/apple-unveils-...



On this day, it turns out there was a big event promoting iphone 11 pro. :https://www.youtube.com/watch?v=-rAeqN-Q7x4

Makes me curious about the prices next day

In [27]:
Apl_stock = pd.read_csv('..\Stock_data\Apple_stock.csv')

In [28]:
Apl_test = Apl_stock.loc[Apl_stock["Date"] <= "2019-09-20"]
Apl_stock_19_09_10 = Apl_test.loc[Apl_test["Date"] >= "2019-09-01"]

In [29]:
#closing prices against date
fig = go.Figure([go.Scatter(x=Apl_stock_19_09_10['Date'], y=Apl_stock_19_09_10['Close'])])
fig.show()


We can see the closing price rising around the date but slowly starts droppint after that. I wonder if the news for next day would display some reasons as to why.

Let's take a look at the news articles for the following day.

In [30]:
Apl.loc[Apl["Date"] == "2019-09-11"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
1371,5015,5015,2019-09-11 02:26:00+00:00,2019-09-11,02:26:00,AAPL,Technology,The Motley Fool,0.000,0.928,0.072,0.3384,The Biggest iPhone News on Tuesday Wasn't the ...,One of the most surprising narratives from the...,https://www.fool.com/investing/2019/09/10/the-...
1372,5017,5017,2019-09-11 05:38:44+00:00,2019-09-11,05:38:44,AAPL,Technology,Reuters,0.067,0.865,0.068,0.2732,Tepid reaction for Apple's new iPhones in Chin...,A lower price tag and new features may not be ...,https://www.reuters.com/article/us-apple-iphon...
1373,5018,5018,2019-09-11 08:30:45+00:00,2019-09-11,08:30:45,AAPL,Technology,The Guardian,0.000,1.000,0.000,0.0000,Apple to launch most expensive iPhone ever in ...,Brexit-battered pound means iPhones 11 Pro and...,https://www.theguardian.com/technology/2019/se...
1374,5019,5019,2019-09-11 08:47:00+00:00,2019-09-11,08:47:00,AAPL,Technology,CNBC,0.215,0.785,0.000,-0.8658,"After being forced to cut prices in China, App...",Apple's new iPhone 11 range commands a premium...,https://www.cnbc.com/2019/09/11/apple-iphone-1...
1375,5020,5020,2019-09-11 09:32:23+00:00,2019-09-11,09:32:23,AAPL,Technology,Reuters,0.174,0.826,0.000,-0.7469,"Apple's new, lower priced iPhone draws tepid r...","Apple Inc's new, lower priced iPhone that com...",https://www.reuters.com/article/us-apple-iphon...
1376,5022,5022,2019-09-11 11:00:41+00:00,2019-09-11,11:00:41,AAPL,Technology,24/7 Wall Street,0.000,1.000,0.000,0.0000,Can Apple Afford to Sell Video for $4.99?,Apple Inc. (NASDAQ: AAPL) has announced its ne...,https://247wallst.com/media/2019/09/11/can-app...
1377,5026,5026,2019-09-11 12:18:00+00:00,2019-09-11,12:18:00,AAPL,Technology,CNBC,0.000,0.963,0.037,0.1603,Apple didn't release its rumored smartwatch sl...,"Apple introduced new iPhones, a new iPad and i...",https://www.cnbc.com/2019/09/11/no-new-apple-w...
1378,5028,5028,2019-09-11 13:03:00+00:00,2019-09-11,13:03:00,AAPL,Technology,Zacks Investment Research,0.072,0.718,0.211,0.4404,Can Competitive Pricing Boost Apple's Revenues?,The tech behemoth has lowered the price of its...,https://www.zacks.com/stock/news/512890/can-co...
1379,5029,5029,2019-09-11 13:19:00+00:00,2019-09-11,13:19:00,AAPL,Technology,CNBC,0.000,1.000,0.000,0.0000,Why Apple made the unusual move to sell its st...,Apple TV+ is cheaper than any other major stre...,https://www.cnbc.com/2019/09/11/the-apple-tv-p...
1380,5030,5030,2019-09-11 13:22:41+00:00,2019-09-11,13:22:41,AAPL,Technology,InvestorPlace,0.137,0.863,0.000,-0.6597,Owners of Apple Stock Should Be Afraid,"With AAPL showing many signs of fear, the owne...",https://investorplace.com/2019/09/owners-of-ap...


While there are many articles on this day with positive sentimenent scores, it is worth noting that there are few articles with negative sentiment scores as well. It seems the new iphone had a price drop. 

Given how people intake news (and given that positively or negatively skewed takes tend to stick out compared to the more neutral takes), I wonder how much of the piture we are missing by combining a given day's sentiment into one total score. (It comes to mind that our overall scores, after combining, were mostly between 0 and 0.3 but these articles with overall positive or negative sentiments alone could have had a significant impact.) Perhaps two more pieces of information we can retain per day per ticker, could be:
1. The highest compound sentiment score an article had about the ticker that day
2. The lowest compound sentiment score an article had about the ticker that day


Let's see what happened on the follwing day as well

In [90]:
Apl.loc[Apl["Date"] == "2019-09-12"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
1412,5084,5084,2019-09-12 11:00:01+00:00,2019-09-12,11:00:01,AAPL,Technology,CNET,0.081,0.919,0.000,-0.2960,The new Apple 10.2-inch iPad is Apple's not-so...,"With a new, larger screen and more pro-level f...",https://www.cnet.com/news/the-new-apple-10-2-i...
1413,5085,5085,2019-09-12 11:14:34+00:00,2019-09-12,11:14:34,AAPL,Technology,Reuters,0.095,0.774,0.131,0.2960,Apple's new iPhones shift smartphone camera ba...,When Apple Inc introduced its triple-camera i...,https://www.reuters.com/article/us-apple-iphon...
1414,5086,5086,2019-09-12 11:30:00+00:00,2019-09-12,11:30:00,AAPL,Technology,The Motley Fool,0.000,1.000,0.000,0.0000,"Here's How Much ""Apple-As-a-Service"" Would Cos...",The monthly fees are starting to add up.,https://www.fool.com/investing/2019/09/12/here...
1415,5089,5089,2019-09-12 12:00:09+00:00,2019-09-12,12:00:09,AAPL,Technology,CNET,0.000,0.920,0.080,0.4215,Apple TV Plus: Our biggest unanswered questions,Apple finally filled in the most glaring blank...,https://www.cnet.com/news/apple-tv-plus-our-bi...
1416,5092,5092,2019-09-12 12:42:57+00:00,2019-09-12,12:42:57,AAPL,Technology,CNN Business,0.205,0.795,0.000,-0.8176,The Apple Watch's secret weapon is its new price,"Almost exactly five years ago, Apple unveiled ...",https://www.cnn.com/2019/09/12/tech/apple-watc...
1417,5097,5097,2019-09-12 14:01:00+00:00,2019-09-12,14:01:00,AAPL,Technology,The Motley Fool,0.065,0.763,0.172,0.5657,The Powerful SaaS Platform No One Is Talking A...,The tech giant is looking more and more like a...,https://www.fool.com/investing/2019/09/12/the-...
1418,5105,5105,2019-09-12 15:01:50+00:00,2019-09-12,15:01:50,AAPL,Technology,TechCrunch,0.000,1.000,0.000,0.0000,Apple tweaks App Store rule changes for childr...,"Originally announced in June, changes to Apple...",https://techcrunch.com/2019/09/12/apple-tweaks...
1419,5115,5115,2019-09-12 15:54:13+00:00,2019-09-12,15:54:13,AAPL,Technology,Market Watch,0.000,0.696,0.304,0.8720,Apple's stock heads for 4th straight gain to 1...,"Shares of Apple Inc. AAPL, +0.98% rose 0.9% in...",https://www.marketwatch.com/story/apples-stock...
1420,5124,5124,2019-09-12 17:00:00+00:00,2019-09-12,17:00:00,AAPL,Technology,The Motley Fool,0.000,0.870,0.130,0.6369,Here's a Rundown of the Biggest Announcements ...,The Cupertino company unveiled a long list of ...,https://www.fool.com/investing/2019/09/12/here...
1421,5125,5125,2019-09-12 17:01:00+00:00,2019-09-12,17:01:00,AAPL,Technology,CNBC,0.000,1.000,0.000,0.0000,Dropcam founder Greg Duffy reportedly leaves A...,Duffy joined Apple in 2017 and was reportedly ...,https://www.cnbc.com/2019/09/12/dropcam-founde...


There are more articles with a negative sentiment, but along with a hoard of positive/neutral articles surrounding them.

Let's look at the total sentiment scores around this day.

In [91]:
#read in the data merged csv
Data_merged = pd.read_csv('..\data\data_merged.csv')

In [93]:
#restrict to apple
Data_Apl = Data_merged.loc[Data_merged["Ticker"] == "AAPL"]

In [96]:
#restrict to the 20-ish days around 2019-09-10

Apl_news_test = Data_Apl.loc[Data_Apl["Date"] <= "2019-09-20"]
Apl_news_19_09_10 = Apl_news_test.loc[Apl_news_test["Date"] >= "2019-09-01"]

In [97]:
#total news sentiment against date
fig = go.Figure([go.Scatter(x=Apl_news_19_09_10['Date'], y=Apl_news_19_09_10['sentiment_tot'])])
fig.show()

In [102]:
#let's look at finvader values
finv = pd.read_csv('..\data\data_finvader.csv')

In [113]:
#finv.head()

In [109]:
Apl_finv = finv.loc[finv["Ticker"] == "AAPL"]
Apl_finv_test = Apl_finv.loc[Apl_finv["Date"] <= "2019-09-20"]
Apl_finv_19_09_10 = Apl_finv_test.loc[Apl_finv_test["Date"] >= "2019-09-01"]


In [112]:
#total finvader news sentiment(s) against date
fig = go.Figure([go.Scatter(x=Apl_finv_19_09_10['Date'], y=Apl_finv_19_09_10['sentiment_tot'])])
fig.show()

Let's compute the mean daily total sentiment

In [119]:
#Apl_finv_merged = Apl_finv.groupby["Date"].mean()
Apl_finv_merged = Apl_finv.groupby('Date')['sentiment_tot'].mean()
#Apl_c = Apl.groupby('Date')['Ticker'].count().reset_index()

In [123]:
Apl_finv_merged = Apl_finv_merged.reset_index()

In [124]:
Apl_finv_merged.head()

,Date,sentiment_tot
0,2019-03-15,0.491367
1,2019-03-18,0.345788
2,2019-03-19,0.453680
3,2019-03-20,0.131575
4,2019-03-21,0.395750


In [126]:
#restricting to the dates around the iphone release announcement
Apl_finv_test2 = Apl_finv_merged.loc[Apl_finv_merged["Date"] <= "2019-09-20"]
Apl_finv_merged_19_09_10 = Apl_finv_test2.loc[Apl_finv_test2["Date"] >= "2019-09-01"]


In [127]:
#total finvader news sentiment (mean) against date
fig = go.Figure([go.Scatter(x=Apl_finv_merged_19_09_10['Date'], y=Apl_finv_merged_19_09_10['sentiment_tot'])])
fig.show()

In [99]:
#trying to plot both graphs on the same grid
#fig = go.Figure([go.Scatter(x=Apl_news_19_09_10['Date'], y=Apl_news_19_09_10['sentiment_tot'])])
#fig = go.Figure([go.Scatter(x=Apl_stock_19_09_10['Date'], y=Apl_stock_19_09_10['Close'])])
#fig.show()

More things we could explore in this vein:
1. Looking at other tickers in this manner
2. Comparing vader scores to finvader scores for other tickers as well


SECTION 2: NVIDIA

In [31]:
Nvd.head()

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
0,46,46,2019-03-19 17:13:49+00:00,2019-03-19,17:13:49,NVDA,Technology,Benzinga,0.0,1.000,0.000,0.0000,Analysts Dissect Nvidia's Product Announcement...,NVIDIA Corporation is holding its GPU tech con...,https://feeds.benzinga.com/~r/benzinga/~3/fNpm...
1,56,56,2019-03-19 20:01:43+00:00,2019-03-19,20:01:43,NVDA,Technology,Market Watch,0.0,0.953,0.047,0.2960,The Ratings Game: Nvidia stock rallies to high...,Nvidia Corp. shares rally following the kickof...,https://feeds.marketwatch.com/~r/marketwatch/i...
2,58,58,2019-03-19 20:08:54+00:00,2019-03-19,20:08:54,NVDA,Technology,Investors Business Daily,0.0,0.767,0.233,0.8442,Nvidia Stock Surges On Data Center Wins From A...,Graphics-chip maker Nvidia announced a flurry ...,https://feedproxy.google.com/~r/BusinessRss/~3...
3,60,60,2019-03-19 21:49:46+00:00,2019-03-19,21:49:46,NVDA,Technology,The Motley Fool,0.0,0.923,0.077,0.4588,Stock Market Today: Investors Applaud NVIDIA's...,NVIDIA used the annual GPU Technology Conferen...,https://www.fool.com/investing/2019/03/19/stoc...
4,66,66,2019-03-20 10:05:00+00:00,2019-03-20,10:05:00,NVDA,Technology,The Motley Fool,0.0,0.828,0.172,0.4404,Better Buy: NVIDIA vs. Cypress Semiconductor,Which chipmaker will keep outperforming the in...,https://www.fool.com/investing/2019/03/20/bett...


In [32]:
Nvd_c = Nvd.groupby('Date')['Ticker'].count().reset_index()

In [34]:
Nvd_counts = Nvd_c.rename(columns={"Ticker": "Article Count"})

In [35]:
Nvd_counts.head()

,Date,Article Count
0,2019-03-19,4
1,2019-03-20,2
2,2019-03-21,1
3,2019-03-22,1
4,2019-03-25,2


Let's check if there are any dates with 0 articles

In [38]:
#number of days with 0 articles
Nvd_counts.loc[Nvd_counts["Article Count"] == 0].count()

Date             0
Article Count    0
dtype: int64

As suspected. How about the days with just one article?

In [39]:
#number of days with 1 article
Nvd_counts.loc[Nvd_counts["Article Count"] == 1].count()

Date             420
Article Count    420
dtype: int64

In [40]:
#number of days with 2 articles
Nvd_counts.loc[Nvd_counts["Article Count"] == 2].count()

Date             263
Article Count    263
dtype: int64

In [43]:
#Histogram of article counts per day
fig = px.histogram(Nvd_counts, x="Article Count",  labels={"Article Count": "Article Count per Day"})
fig.update_layout(yaxis_title="Days")
fig.show()

Let's look at the article count per month

In [47]:
#Create a column for month
Nvd_counts = Nvd_counts.assign(month = Nvd_counts.Date.str[0:7])

#create another dataframe containing article count by month
Nvd_counts_month = Nvd_counts.groupby('month')['Article Count'].sum().reset_index()

In [48]:
Nvd_counts_month.head()

,month,Article Count
0,2019-03,12
1,2019-04,7
2,2019-05,45
3,2019-06,35
4,2019-07,38


In [50]:
#Histogram of article counts per month
fig = px.histogram(Nvd_counts_month, x="Article Count",  labels={"Article Count": "Article Count per Month"})
fig.update_layout(yaxis_title="Months")
fig.show()

Let's look at article counts against date

In [55]:
fig = go.Figure([go.Scatter(x=Nvd_counts['Date'], y=Nvd_counts['Article Count'])])
fig.show()


There seem to be a considerable amount of stretches (some few days long, some a week-ish), where there was only one article published per day. Let's look at a few from one of those strectches and a few isolated ones.

There seems to be one such strech during 2021-01-14 - 2021-02-08

In [61]:
Nvd_temp = Nvd.loc[Nvd["Date"] >= "2021-01-14"]
Nvd_temp_2 = Nvd_temp.loc[Nvd_temp["Date"] <= "2021-02-08"]

In [62]:
#Print the table for the dates 2021-01-14 - 2021-02-08
Nvd_temp_2

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
994,22376,22376,2021-01-14 18:00:52+00:00,2021-01-14,18:00:52,NVDA,Technology,Benzinga,0.024,0.783,0.193,0.8625,Nvidia's Comprehensive Involvement In Gaming M...,NVIDIA Corporation (NASDAQ: NVDA) is a dominan...,https://www.benzinga.com/analyst-ratings/analy...
995,22453,22453,2021-01-16 12:43:15+00:00,2021-01-16,12:43:15,NVDA,Technology,ETF Trends,0.070,0.877,0.052,-0.1796,Why Nvidia Is My “Slam Dunk” Investment for th...,What was the most important financial event of...,https://www.etftrends.com/tactical-allocation-...
996,22460,22460,2021-01-17 13:00:00+00:00,2021-01-17,13:00:00,NVDA,Technology,The Motley Fool,0.136,0.796,0.068,-0.4201,Is NVIDIA About to Bring More Pain to AMD?,NVIDIA's dominance in the graphics card busine...,https://www.fool.com/investing/2021/01/17/is-n...
997,22472,22472,2021-01-18 15:06:56+00:00,2021-01-18,15:06:56,NVDA,Technology,InvestorPlace,0.095,0.835,0.069,-0.0258,Nvidia: Can Anything Stop This Train?,Nvidia stock has stalled since it announced a ...,https://investorplace.com/2021/01/nvidia-can-a...
998,22542,22542,2021-01-20 16:05:00+00:00,2021-01-20,16:05:00,NVDA,Technology,The Motley Fool,0.129,0.554,0.317,0.7269,"Forget Bitcoin, This Hot Tech Stock Is a Bette...",There's a safer way to profit from the cryptoc...,https://www.fool.com/investing/2021/01/20/forg...
999,22664,22664,2021-01-23 13:45:00+00:00,2021-01-23,13:45:00,NVDA,Technology,The Motley Fool,0.000,0.723,0.277,0.7845,My Best 5G Stock for 2021,This GPU powerhouse will provide key technolog...,https://www.fool.com/investing/2021/01/23/my-b...
1000,23090,23090,2021-02-02 14:00:00+00:00,2021-02-02,14:00:00,NVDA,Technology,Seeking Alpha,0.000,0.741,0.259,0.7717,Nvidia: Arm Acquisition Will Increase Share Pr...,Nvidia acquisition of Arm for $40 bln will com...,https://seekingalpha.com/article/4402456-nvidi...
1001,23295,23295,2021-02-04 12:15:00+00:00,2021-02-04,12:15:00,NVDA,Technology,The Motley Fool,0.000,1.000,0.000,0.0000,The Long-Term Potential for NVIDIA and ARM Is ...,NVIDIA's acquisition of ARM is all about provi...,https://www.fool.com/investing/2021/02/04/long...
1002,23407,23407,2021-02-07 13:42:00+00:00,2021-02-07,13:42:00,NVDA,Technology,The Motley Fool,0.000,0.732,0.268,0.7715,What to Watch When NVIDIA Reports Earnings,Certain parts of NVIDIA's business performed v...,https://www.fool.com/investing/2021/02/07/what...
1003,23432,23432,2021-02-08 21:10:19+00:00,2021-02-08,21:10:19,NVDA,Technology,Investors Business Daily,0.000,0.930,0.070,0.3400,"AI Technology, Autonomous Vehicle Leader Nvidi...","As top funds scoop up semiconductor stocks, AI...",https://www.investors.com/research/how-to-find...


When I first saw this atleast one article per day behaviour I assumed it was one outlet publishig a daily obligatory article about a given ticker. But looking at the above stretch of time this seems to not be the case.

One other notable thing is that these don't seem to be neutral sounding reports of mere stock movement.

The other interesting thing I noticed was this article: https://www.nasdaq.com/articles/why-nvidia-is-my-slam-dunk-investment-for-the-decade-2021-01-16
that vader gave an overall negative sentiment to, despite being a very positive article on Nvidia.
I wonder if this is because, the headline contains the word "dunk" and the text contains sentenses like "But they were wrong" and "But if we woke up tomorrow to find NVIDIA (NVDA) had disappeared, the world would be a much worse place". (We should look at the Finvader sentiment score for this article)

In [63]:
#Let's look at an isolated date with only one article
Nvd.loc[Nvd["Date"] == "2021-06-16"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
1260,27746,27746,2021-06-16 18:42:06+00:00,2021-06-16,18:42:06,NVDA,Technology,Seeking Alpha,0.0,1.0,0.0,0.0,NVIDIA Corporation (NVDA) Management Presents ...,NVIDIA Corporation (NVDA) Management Presents ...,https://seekingalpha.com/article/4435201-nvidi...


Let's take a look at the date with the highest number of articles in our training set

In [69]:
Nvd_test_3 = Nvd_counts.loc[Nvd["Date"] <= "2023-03-14"]

In [71]:
Nvd_test_3["Article Count"].max()

66

In [72]:
Nvd_test_3.loc[Nvd_test_3["Article Count"] == 66]

,Date,Article Count,month
1031,2023-05-25,66,2023-05


In [73]:
Nvd.loc[Nvd["Date"] == "2023-05-25"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
3201,51843,51843,2023-05-25 00:37:11+00:00,2023-05-25,00:37:11,NVDA,Technology,The Motley Fool,0.000,1.000,0.000,0.0000,Nvidia Stock Rockets 25% After Earnings -- NVD...,Nvidia stock surged over 25% on a double earni...,https://www.fool.com/investing/2023/05/24/nvid...
3202,51844,51844,2023-05-25 01:15:00+00:00,2023-05-25,01:15:00,NVDA,Technology,Market Watch,0.000,0.916,0.084,0.4310,Nvidia CFO on record-breaking forecast: ‘The i...,Since ChatGPT grabbed the world's attention la...,https://www.marketwatch.com/story/nvidia-cfo-o...
3203,51845,51845,2023-05-25 02:20:00+00:00,2023-05-25,02:20:00,NVDA,Technology,The Motley Fool,0.050,0.898,0.052,0.0258,Nvidia Stock Soars 25% as AI-Powered Guidance ...,"Fiscal Q1 revenue declined 13% year over year,...",https://www.fool.com/investing/2023/05/24/nvda...
3204,51846,51846,2023-05-25 02:33:03+00:00,2023-05-25,02:33:03,NVDA,Technology,Seeking Alpha,0.000,0.948,0.052,0.4939,NVIDIA Corp. (NVDA) Q1 2024 Earnings Call Tran...,NVIDIA Corp. (NASDAQ:NVDA ) Q1 2024 Earnings C...,https://seekingalpha.com/article/4607199-nvidi...
3205,51848,51848,2023-05-25 06:10:47+00:00,2023-05-25,06:10:47,NVDA,Technology,Fox Business,0.000,1.000,0.000,0.0000,Nvidia leading AI charge,The rise of AI and machine learning has thrust...,https://www.foxbusiness.com/markets/nvidia-lea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3262,51930,51930,2023-05-25 20:36:00+00:00,2023-05-25,20:36:00,NVDA,Technology,WSJ,0.000,1.000,0.000,0.0000,Chip maker Nvidia Powers Nasdaq Higher,The Dow slips even as the artificial-intellige...,https://www.wsj.com/articles/global-stocks-mar...
3263,51931,51931,2023-05-25 20:42:00+00:00,2023-05-25,20:42:00,NVDA,Technology,Market Watch,0.000,0.933,0.067,0.5267,Nvidia's market capitalization gain on Thursda...,"On Thursday, Nvidia's stock took an epic jump ...",https://www.marketwatch.com/story/nvidias-mark...
3264,51932,51932,2023-05-25 20:58:00+00:00,2023-05-25,20:58:00,NVDA,Technology,Market Watch,0.042,0.758,0.199,0.8360,Nvidia's 24% jump Thursday lit an AI fire that...,Nvidia Corp. did something remarkable to set u...,https://www.marketwatch.com/story/nvidias-24-j...
3265,51935,51935,2023-05-25 21:31:21+00:00,2023-05-25,21:31:21,NVDA,Technology,Investors Business Daily,0.000,1.000,0.000,0.0000,Nasdaq Surges As Nvidia Rockets 24% To Record ...,"The stock market ended mixed Thursday, as the ...",https://www.investors.com/market-trend/the-big...


In [76]:
Nvd_test_4 = Nvd.loc[Nvd["Date"] == "2023-05-25"]


In [78]:
Nvd_test_4["sentiment_tot"].mean()

0.39790909090909093

Given that our sentiment totals were mostly gathered in the range 0 - 0.2, this total sentiment is extremely impressive. Let's see how this compares to stock price movement.

In [85]:
#Read in Nvidia Stock csv
Nvd_stock = pd.read_csv('..\Stock_data\please_read_Nvidia_stock.csv')

In [86]:
Nvd_stock.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-03-01 00:00:00-05:00,38.802182,39.268987,38.218670,38.846874,35718800,0.0,0.0
1,2019-03-04 00:00:00-05:00,39.231731,39.460169,38.362674,38.928802,40999200,0.0,0.0
2,2019-03-05 00:00:00-05:00,38.772368,39.246626,38.228590,38.864243,36292400,0.0,0.0
3,2019-03-06 00:00:00-05:00,38.774867,38.859289,37.727032,37.754345,40352400,0.0,0.0
4,2019-03-07 00:00:00-05:00,37.518452,37.615289,36.599731,37.061573,45010400,0.0,0.0


In [87]:
Nvd_stock_test = Nvd_stock.loc[Nvd_stock["Date"] <= "2023-06-05"]
Nvd_stock_23_05_25 = Nvd_stock_test.loc[Nvd_stock_test["Date"] >= "2023-05-15"]

In [88]:
#closing prices against date
fig = go.Figure([go.Scatter(x=Nvd_stock_23_05_25['Date'], y=Nvd_stock_23_05_25['Close'])])
fig.show()


Note that as expected the closing price of Nvidia stock on the 25th is much higher than that of the day before. The price still keeps gradually rising after that until the 30th and then falls a bit on the 31st. Let's see what was on the news on the 31st.

In [89]:
Nvd.loc[Nvd["Date"] == "2023-05-31"]

,index,Unnamed: 0,Date & Time,Date,Time,Ticker,Sector,Source,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_tot,Headline,Text,URL
3360,52101,52101,2023-05-31 00:00:27+00:00,2023-05-31,00:00:27,NVDA,Technology,Seeking Alpha,0.000,0.895,0.105,0.7184,Nvidia: More Upside Likely Ahead,Nvidia Corporation soared following strong gui...,https://seekingalpha.com/article/4608279-nvidi...
3361,52105,52105,2023-05-31 08:00:00+00:00,2023-05-31,08:00:00,NVDA,Technology,Business Insider,0.000,1.000,0.000,0.0000,This video shows how Nvidia's AI could transfo...,Nvidia debuted a new AI tool at Computex Taipe...,https://www.businessinsider.com/nvidia-ai-tran...
3362,52107,52107,2023-05-31 09:20:00+00:00,2023-05-31,09:20:00,NVDA,Technology,The Motley Fool,0.000,0.860,0.140,0.7579,"1 Reason to Sell Nvidia Right Now, but 1 Reaso...",Nvidia paused its share repurchase plan last q...,https://www.fool.com/investing/2023/05/31/1-re...
3363,52110,52110,2023-05-31 10:15:00+00:00,2023-05-31,10:15:00,NVDA,Technology,The Motley Fool,0.000,0.882,0.118,0.3400,1 Green Flag for Nvidia Stock Investors,Nvidia's stock price is soaring as investors g...,https://www.fool.com/investing/2023/05/31/1-gr...
3364,52111,52111,2023-05-31 10:16:00+00:00,2023-05-31,10:16:00,NVDA,Technology,WSJ,0.085,0.845,0.070,-0.1280,Have Megafirms Like Nvidia Grown Too Big? Inve...,"Generative AI, banking troubles and a volatile...",https://www.wsj.com/articles/have-megafirms-li...
3365,52112,52112,2023-05-31 10:45:00+00:00,2023-05-31,10:45:00,NVDA,Technology,The Motley Fool,0.000,1.000,0.000,0.0000,Nvidia Crossed a $1 Trillion Valuation -- How ...,Can Nvidia meet analysts' expectations for the...,https://www.fool.com/investing/2023/05/31/nvid...
3366,52113,52113,2023-05-31 10:45:48+00:00,2023-05-31,10:45:48,NVDA,Technology,InvestorPlace,0.000,0.903,0.097,0.4767,Is NVDA Stock Destined to Be the Next Trillion...,Nvidia (NASDAQ: NVDA ) stock impressed with it...,https://investorplace.com/2023/05/is-nvda-stoc...
3367,52115,52115,2023-05-31 11:15:00+00:00,2023-05-31,11:15:00,NVDA,Technology,Seeking Alpha,0.099,0.724,0.177,0.7430,"11 Billion Reasons To Buy Nvidia, And 2.2 Tril...",Nvidia Corporation's blowout earnings report a...,https://seekingalpha.com/article/4608256-11-bi...
3368,52118,52118,2023-05-31 12:15:00+00:00,2023-05-31,12:15:00,NVDA,Technology,Business Insider,0.000,0.872,0.128,0.5106,Nvidia has AI to thank for entering the $1 tri...,Nvidia's market capitalization hit $1 trillion...,https://markets.businessinsider.com/news/stock...
3369,52122,52122,2023-05-31 13:13:29+00:00,2023-05-31,13:13:29,NVDA,Technology,The Motley Fool,0.020,0.828,0.152,0.7430,1 Artificial Intelligence Growth Stock That Wa...,Nvidia is the gold standard in workstation gra...,https://www.fool.com/investing/2023/05/31/1-ai...


There seem to be two articles with negative sentiment displayed, but everything else is positive.

One other possible take away from this is that, in addition to the daily total sentiment, we may want to consider the daily total sentiment as a percentage of the average sentiment of one week (or one month), till that date. This was we're keeping track of the mild changes made that day to the overall sentiment that the traders/readers have already aclamatized themselves to.

In the interest of checking our biases, we should look at the news published on the two days inbetween the "big rise" and the "mild fall" as well. (To be continued...)